# Big Data Analytics: NYC Crashes

## Einleitung

In den Straßen New York Citys strotzt es gerade so vor Verkehr, wodurch Verkehrsunfälle an der Tagesordnung stehen. Durch das NYC Open Data Project werden die erfassten Verkehrsunfälle für die Allgemeinheit zugänglich.

Thema des Berichts und unseres Projekts ist die Nutzung dieser öffentlich zugänglichen Unfalldaten zur Analyse und visuellen Auswertung. 

### Aufbau der Data Pipeline

Für das Projekt soll eine Data-Pipeline aufgebaut werden, die die Daten in die Datenbank lädt. 
Von dort aus sollen die Daten von einem Python-Script zur Analyse angefragt werden.

Beteiligt an der Data-Pipeline ist eine Kafka-Instanz, wobei ein Python-Script einen Producer dafür darstellt und ein weiteres Python-Script einen Consumer. Das Producer-Script lädt die Daten zeilenweise aus der Datenquelle, einer CSV-Datei, in Kafka ein, während der Consumer die Daten aus Kafka ausliest und in die MongoDB schreibt. Das Analyse-Script bezieht die Daten wiederum aus der MongoDB.

#### Installation der benötigten Docker-Container

Das Script `start_docker.sh` startet den MongoDB Docker Container, beziehungsweise erstellt bei Bedarf einen neuen Docker Container. Dabei wird, falls noch kein Image für die DockerDB vorhanden ist, das neueste Heruntergeladen. 

Die weiteren benötigten Docker Container sind in der Docker-Compose-Konfiguration (`docker-compose.yml`) vorhanden.
Diese können mit `docker-compose up` erstellt oder heruntergeladen werden. 
Mit `docker-compose down` können die gestarteten Container gestoppt werden, wobei auf die Vollendigung des Befehls gewartet werden sollte.

Auf manchen Systemen befindet sich die Docker Engine nach stoppen der Docker Container mit `docker-compose down` in einem nicht-revertierbarem Fehlerzustand, wobei ein _Neustart des Computers_ unausweichlich ist. Weitere Informationen zu dem Fehler, wobei jedoch _keine der angegebenen Workarounds genutzt werden sollte_, findet sich [auf GitHub](https://github.com/docker/for-linux/issues/162). 

### Datenquelle
Als Datenquellen dienen die vom NYC Open Data Project bereitgestellten Unfalldaten des NYPD. Die detaillierten Unfalldaten befinden sich in 3 unterschiedlichen Datenquellen die unter Anderem als CSV-Datei verfügbar sind: [Motor Vehicle Collisions](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=Motor+Vehicle+Collisions&q=crashes).

Enthalten in den Daten sind die am Unfall beteiligten Verkehrsteilnehmer, die Verletzten und Toten, sowie die genaue Position des Unfalls. Darüberhinaus sind noch weitere Daten in den Quellen enthalten, auf die teilweise im Analyseabschnitt genauer eingegangen wird.

### Analyseziele
Ziel der Analyse der Daten ist herauszufinden, welche saisonalen und lokalen Zusammenhänge festzustellen sind, wodurch sich besondere Hotspots im Stadtraum New Yorks lokalisieren lassen. Des weiteren soll ermittelt werden, welche Verkehrsteilnehmer besonders oft getötet oder verletzt werden, sowie an welchen Stellen sich gerade schwerwiegende Unfälle häufen. Anhand einer animierten Karte kann dadurch das Unfallgeschehen über die Zeit betrachtet werden.

### Installation der Python Packages

In [ ]:
pip install kafka-python pymongo

### Importieren benötigter Module

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path
from multiprocessing import Process
from time import sleep
from data.schemas import schemas
import platform
windows = True if 'Windows' in platform.system() else False 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Herunterladen der Datensätze
Zum herunterladen der Datensätze werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien die die Unfalldaten zeilenweise enthalten.

In [ ]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

## Einlesen der Daten und senden über den Producer

### Produzieren der CSV-Daten in Kafka

In [ ]:
def send_to_kafka(topic):
    print(f'Started {topic} Producer process.')   
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
    
    with open((fn:=Path('data') / f'{topic}.csv')) as source:
        for i, row in enumerate(source.readlines()):
            if i:
                producer.send(f'nyc_{topic}', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))
            if not i % 100000 and i:
                print(f'CSV → Kafka: Read {i} lines from {fn}.')
    producer.close()
    print(f'Producer for topic {topic} closed after {i} entries.')
                
producers = {
    'crashes': Process(target=send_to_kafka, args=['crashes']),
    'vehicles': Process(target=send_to_kafka, args=['vehicles']),
    'persons': Process(target=send_to_kafka, args=['persons']),
}

### Konsumieren der Daten und Import in die Datenbank

Zunächst werden die Daten aus den Topics `nyc_persons` und `nyc_vehicles` verarbeitet.

In [ ]:
def process_topic(topic, types):
    with  MongoClient("mongodb://localhost:27017") as client:
        target_db = client['nyc_crashes'][topic]
        counter = 0

        consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest')

        consumer.subscribe([f'nyc_{topic}'])

        print(f'Started {topic} Consumer process.')
        for row in consumer:
            row = row.value.decode('utf-8').split(',')

            counter += 1
            if counter and not counter % 100000:
                print(f'Kafka → MongoDB: Read {counter} lines from Kafka topic {topic}')
            res = {}
            for idx, (db_field, field_type) in enumerate(types.items()):
                try:
                    if row_data := row[idx]:
                        res[db_field] = field_type.__call__(row_data) if not isinstance(row_data, field_type) else row_data
                    else:
                        res[db_field] = None   
                except Exception as e:
                    res[db_field] = None

            # make sure the document is identifiable
            if res['_id']:
                try:
                    target_db.insert_one(res)
                except Exception as e:
                    pass
            
        print(f'Finished consuming {topic}!')

            
consumers = {
    'vehicles': Process(target=process_topic, args=(
        'vehicles', schemas['vehicles']
    )),
    'persons': Process(target=process_topic, args=(
        'persons', schemas['persons']
    ))
}

### Starten der Prozesse
Nun werden die Prozesse zum Produzieren der Daten aus `vehicles.csv`, `persons.csv`, und `crashes.csv` und die Prozesse zum Konsumieren der Topics `nyc_vehicles` und `nyc_persons` gestartet.

In [ ]:
if windows:
    send_to_kafka('vehicles')
    process_topic('vehicles', schemas['vehicles'])
    send_to_kafka('persons')
    process_topic('persons', schemas['persons'])
    send_to_kafka('crashes')
else:
    for topic in 'persons vehicles crashes'.split(' '):
        producers[topic].start()
    for topic in 'persons vehicles'.split(' '):
        consumers[topic].start()

Danach können diese Daten mit den Daten aus `nyc_crashes` gejoined werden.

In [ ]:
def process_crashes():
    with  MongoClient("mongodb://localhost:27017") as client:
        target_db = client['nyc_crashes'][topic]
        consumer = KafkaConsumer('nyc_crashes', bootstrap_servers=['localhost:9092'], auto_offset_reset="earliest")
        counter = 0

        for row in consumer: 
            row = row.value.decode('utf-8').split(',')
            counter += 1
            if counter and not counter % 100000:
                print(counter)
            res = {}
            for idx, (db_field, field_type) in enumerate(schemas['crashes'].items()):
                try:
                    if row_data := row[idx]:
                        res[db_field] = field_type.__call__(row_data) if not isinstance(row_data, field_type) else row_data
                    else:
                        res[db_field] = None
                except Exception as e:
                    res[db_field] = None

            # make sure the document is identifiable
            if res['_id']:
                res['vehicles'] = list(db_by_topic['vehicles'].find({'_id': {'$eq': res['_id']}}))
                res['persons'] = list(db_by_topic['persons'].find({'_id': {'$eq': res['_id']}}))

                db_by_topic['crashes'].insert_one(res)

        print(f'Finished consuming {topic}!')

consumers['crashes'] = Process(target=process_crashes)

### Starten des Join-Prozesses
Im letzten Schritt muss der Prozess gestartet werden, der das joinen der beiden Tabellen `vehicles` und `persons` in die Daten des Topics `nyc_crashes` durchführt.

In [ ]:
assert False, 'breakpoint'

In [ ]:
if windows:
    process_crashes()
else:
    consumers['crashes'].start()